In [22]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [23]:
import pandas as pd

query = "SELECT * FROM disclosure_events"
df = pd.read_sql(query, engine)
print(f"데이터베이스에서 조회된 공시 이벤트 수: {len(df)}")
df.head()


DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


데이터베이스에서 조회된 공시 이벤트 수: 31165


,id,stock_code,market,company_name,report_name,disclosed_at,summary_kr,raw,created_at,ret_1m,ret_3m,ret_10m,ret_15m,ret_30m,ret_60m,label,score
0,52054,KR7068270008,KOSPI,셀트리온,투자판단관련주요경영사항(CT-P42(아일리아 바이오시밀러) 임상 3상 결과 발표),2023-04-02 23:00:06+00:00,셀트리온은 당뇨병성 황반부종 환자를 대상으로 한 아일리아 바이오시밀러(CP-T42)...,2023.04.03 08:00:05\n기업명: 셀트리온(시가총액: 21조 948억)...,2025-08-09 19:13:27.639981,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,32519,KR7114190002,KOSDAQ,강원에너지,타법인주식및출자증권취득결정(자율공시),2022-07-01 01:03:17+00:00,"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...","2022.07.01 10:03:16\n기업명: 강원에너지(시가총액: 1,842억)\...",2025-08-09 19:13:15.234304,NaN,NaN,NaN,NaN,NaN,NaN,neutral,0.9723
2,33989,None,None,CBI,주요사항보고서(무상증자결정),2022-07-29 07:26:53+00:00,"CBI는 2022년 7월 29일 무상증자를 결정하였으며, 발행일은 9월 6일이고 기...","2022.07.29 16:26:52\n기업명: CBI(시가총액: 1,681억)\n보...",2025-08-09 19:13:15.234304,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,64741,KR7381970003,KOSPI,케이카,영업(잠정)실적(공정공시),2023-11-01 23:57:28+00:00,"케이카는 2023년 3분기 매출액 5,286억, 영업이익 184억, 순이익 54억을...","2023.11.02 08:57:27\n기업명: 케이카(시가총액: 4,727억)\n보...",2025-07-08 17:09:01.716527,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,32520,KR7082920000,KOSDAQ,비츠로셀,단일판매ㆍ공급계약체결,2022-07-01 01:24:45+00:00,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,"2022.07.01 10:24:44\n기업명: 비츠로셀(시가총액: 2,477억)\n...",2025-08-09 19:13:15.234304,NaN,NaN,NaN,NaN,NaN,NaN,positive,0.9959


In [24]:
# market 종류와 각각의 비율 분석
market_counts = df['market'].value_counts()
print("시장별 공시 분포:")
print(market_counts)
print(f"\n총 공시 건수: {len(df)}")
print("\n시장별 비율:")
for market, count in market_counts.items():
    ratio = count / len(df) * 100
    print(f"{market}: {count}건 ({ratio:.1f}%)")


시장별 공시 분포:
market
KOSDAQ           13403
KOSPI            12392
KOSDAQ GLOBAL     1057
Name: count, dtype: int64

총 공시 건수: 31165

시장별 비율:
KOSDAQ: 13403건 (43.0%)
KOSPI: 12392건 (39.8%)
KOSDAQ GLOBAL: 1057건 (3.4%)


In [25]:
# disclosed_at을 datetime으로 변환 (이미 timezone-aware 상태)
df['disclosed_at'] = pd.to_datetime(df['disclosed_at'])

# 이미 timezone-aware이므로 tz_convert만 사용하여 KST로 변환
df['disclosed_at_kst'] = df['disclosed_at'].dt.tz_convert('Asia/Seoul')

# KST 시간에 따라 데이터프레임 정렬
df = df.sort_values('disclosed_at_kst')

# 시간만 추출 (KST 기준)
df['time_only'] = df['disclosed_at_kst'].dt.time



# 장중 시간 정의 (9:00 ~ 15:30)
import datetime
market_open = datetime.time(9, 0)
market_close = datetime.time(15, 30)

# 장중/장외 분류
df['market_session'] = df['time_only'].apply(
    lambda x: '장중' if market_open <= x <= market_close else '장외'
)

# 결과 집계
session_counts = df['market_session'].value_counts()
print("공시 발표 시간대별 분포:")
print(session_counts)
print(f"\n장중 비율: {session_counts['장중'] / len(df) * 100:.1f}%")
print(f"장외 비율: {session_counts['장외'] / len(df) * 100:.1f}%")


공시 발표 시간대별 분포:
market_session
장외    19745
장중    11420
Name: count, dtype: int64

장중 비율: 36.6%
장외 비율: 63.4%


In [26]:
import pandas as pd

df = df[['company_name', 'disclosed_at_kst', 'ret_1m', 'ret_3m', 'ret_10m', 'ret_60m']]

df

,company_name,disclosed_at_kst,ret_1m,ret_3m,ret_10m,ret_60m
13431,,2022-06-01 11:01:00+09:00,NaN,NaN,NaN,NaN
13432,,2022-06-01 11:01:56+09:00,NaN,NaN,NaN,NaN
13433,,2022-06-01 11:02:49+09:00,NaN,NaN,NaN,NaN
13434,넥스턴바이오,2022-06-02 08:00:08+09:00,NaN,NaN,NaN,NaN
13435,에어부산,2022-06-02 08:00:09+09:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2448,에치에프알,2023-12-29 15:56:21+09:00,NaN,NaN,NaN,NaN
2449,광동제약,2023-12-29 16:22:38+09:00,NaN,NaN,NaN,NaN
2450,효성첨단소재,2023-12-29 16:43:03+09:00,NaN,NaN,NaN,NaN
2442,에스원,2023-12-29 16:51:25+09:00,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

table = "disclosure_events"
query = f"""SELECT * FROM {table}
WHERE stock_code is not NULL AND market IN ('KOSPI', 'KOSDAQ', 'KOSDAQ GLOBAL') AND (
                  (EXTRACT(HOUR FROM disclosed_at AT TIME ZONE 'Asia/Seoul') BETWEEN 9 AND 14)
                  OR
                  (EXTRACT(HOUR FROM disclosed_at AT TIME ZONE 'Asia/Seoul') = 15
                   AND EXTRACT(MINUTE FROM disclosed_at AT TIME ZONE 'Asia/Seoul') <= 30)
              )
ORDER BY id ASC"""
target_df = pd.read_sql(query, engine)

target_df

데이터베이스에서 조회된 공시 이벤트 수: 10044


,id,stock_code,market,company_name,report_name,disclosed_at,summary_kr,raw,created_at,ret_1m,ret_3m,ret_10m,ret_15m,ret_30m,ret_60m,label,score
0,31437,KR7002990000,KOSPI,금호건설,단일판매ㆍ공급계약체결,2022-06-02 00:26:33+00:00,금호건설은 센트럴엔49피에프브이 주식회사와 남양주 센트럴N49 개발사업의 건설공사 ...,"2022.06.02 09:26:32\n기업명: 금호건설(시가총액: 3,415억)\n...",2025-08-09 19:13:13.499385,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,31438,KR7145020004,KOSDAQ GLOBAL,휴젤,주식등의대량보유상황보고서(약식),2022-06-02 00:27:51+00:00,"휴젤의 주식 보유 지분이 6.06%에서 7.06%로 증가했으며, 이번 보고서에서는 ...","2022.06.02 09:27:50\n기업명: 휴젤(시가총액: 1조 5,370억)\...",2025-08-09 19:13:13.499385,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,31439,KR7321550006,KOSDAQ,티움바이오,기타경영사항(자율공시)(MSD와 공동임상 및 의약품 지원 계약 체결),2022-06-02 00:54:03+00:00,"티움바이오가 MSD와 공동임상 및 의약품 지원 계약을 체결했으며, TGF-β와 VE...","2022.06.02 09:54:02\n기업명: 티움바이오(시가총액: 3,920억)\...",2025-08-09 19:13:13.499385,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,31441,KR8702070002,KOSPI,프레스티지바이오파마,주식등의대량보유상황보고서(일반),2022-06-02 01:50:39+00:00,프레스티지바이오파마의 최대주주인 메이슨파트너스는 2022년 6월 현재 지분율이 51...,"2022.06.02 10:50:38\n기업명: 프레스티지바이오파마(시가총액: 7,9...",2025-08-09 19:13:13.499385,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,31442,KR7293780003,KOSDAQ,압타바이오,투자판단관련주요경영사항(조영제유발급성신장손상(CI-AKI) 치료제 Isuzinaxi...,2022-06-02 02:08:36+00:00,압타바이오는 미국 FDA에 조영제유발 급성 신장손상 치료제인 Isuzinaxib(A...,"2022.06.02 11:08:35\n기업명: 압타바이오(시가총액: 4,092억)\...",2025-08-09 19:13:13.499385,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10039,68168,KR7068270008,KOSPI,셀트리온,투자판단관련주요경영사항(CT-P41(프롤리아&엑스지바 바이오시밀러) 한국 품목허가 신청),2023-12-29 06:08:32+00:00,셀트리온은 프롤리아와 엑스지바 바이오시밀러인 CT-P41의 한국 품목허가를 신청하였...,"2023.12.29 15:08:30\n기업명: 셀트리온(시가총액: 29조 5,002...",2025-07-08 17:09:03.053194,NaN,NaN,NaN,NaN,NaN,NaN,positive,0.9998
10040,68169,KR7060570009,KOSDAQ,드림어스컴퍼니,기타경영사항(자율공시)(전환우선주의 보통주 전환청구),2023-12-29 06:09:37+00:00,"드림어스컴퍼니는 2021년 6월 16일에 발행된 전환우선주 13,135,673주가 ...","2023.12.29 15:09:36\n기업명: 드림어스컴퍼니(시가총액: 1,669억...",2025-07-08 17:09:03.053194,NaN,NaN,NaN,NaN,NaN,NaN,neutral,0.9974
10041,68170,KR7247540008,KOSDAQ GLOBAL,에코프로비엠,주식등의대량보유상황보고서(일반),2023-12-29 06:11:26+00:00,에코프로비엠은 경영권 영향 목적으로 보유주식을 50.91%에서 50.34%로 줄였으...,"2023.12.29 15:11:25\n기업명: 에코프로비엠(시가총액: 28조 1,6...",2025-07-08 17:09:03.053194,NaN,NaN,NaN,NaN,NaN,NaN,negative,0.6860
10042,68171,KR7452260003,KOSPI,한화갤러리아,조회공시요구(현저한시황변동)에대한답변(중요공시대상없음),2023-12-29 06:17:19+00:00,"한화갤러리아는 최근 시황변동과 관련하여 중요 공시사항이 없음을 밝혔으며, 지난 12...","2023.12.29 15:17:18\n기업명: 한화갤러리아(시가총액: 2,633억)...",2025-07-08 17:09:03.053194,NaN,NaN,NaN,NaN,NaN,NaN,neutral,0.9998
